SQL Databases
=============

There is a standard API that Python uses to communicates with SQL databases.  We'll work with the SQLite database, since that comes bundled with Python, but working with other databases should be very similar, although there may be slight differences in the actual SQL commands sent.

In [ ]:
import sqlite3 as db

The first thing to do is to connect to the database.  For SQLite, the command looks like this:

In [ ]:
connection = db.connect("my_database.sqlite")

Different databases have different options for the connect command.  For example, the `cx-oracle` database module might connect with:

    connection = db.connect(username, password, host, port,  'XE')

Once you have a connection, you usually want to create a cursor object which can execute commands provided as strings:

In [ ]:
cursor = connection.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS orders(
        order_id TEXT PRIMARY KEY,
        date TEXT,
        symbol TEXT,
        quantity INTEGER,
        price NUMBER)""")
cursor.execute("""INSERT INTO orders VALUES
        ('A0001', '2013-12-01', 'AAPL', 1000, 203.4)""")
connection.commit()

You don't want to be building SQL commands using string operations however (that's how you get SQL injection attacks!), so the database modules provide ways of parametrizing commands:

In [ ]:
orders = [
          ("A0002","2013-12-01","MSFT",1500,167.5),
          ("A0003","2013-12-02","GOOG",1500,167.5)
]
cursor.executemany("""INSERT INTO orders VALUES
    (?, ?, ?, ?, ?)""", orders)
connection.commit()

Note that different databases may indicate parameters using different symbols.  For example, the `cx-oracle` database module would use this syntax:

    cursor.executemany("""INSERT INTO orders VALUES
        (:order_id, :date, :symbol, :quantity, :price)""",
        orders)

The supported styles for a database module are given by `db.paramstyle`:

In [ ]:
db.paramstyle

As you may have noticed, queries which modify the database must be committed after all operations have been performed.  If you don't commit, queries will not see the modifications to the database.

If you wish to stop a commit, the `rollback()` method allows you to clean-up and restore the state of the database to the last `commit()`.  This is commonly used to safely handle errors and exceptions during database operations.

A common pattern looks like this:

    try:
        ... # perform some operations
    except:
        connection.rollback()
        raise
    else:
        connection.commit()


To extract data from a database, you execute a SELECT command and then use the cursor as an iterator over the rows returned.

In [ ]:
stock = 'MSFT'
cursor.execute("""SELECT *
    FROM orders
    WHERE symbol=?
    ORDER BY quantity""", (stock,))
for row in cursor:
    print row

The `cursor.fetchone()` method can be used to get the next row, and the `cursor.fetchall()` method returns a list of all the rows (which could be very large!).

In [ ]:
stock = 'AAPL'
cursor.execute("""SELECT *
    FROM orders
    WHERE symbol=?
    ORDER BY quantity""", (stock,))
cursor.fetchall()

Finally, once you are done you can close the cusor and connection:

In [ ]:
cursor.close()
connection.close()

Copyright 2013-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com